In [1]:
import pandas as pd
import numpy as np

In [3]:
survey_raw = pd.read_excel('survey.xlsx', index_col=0) 
travel_raw = pd.read_excel('travel.xlsx') 

/Users/peterslina/Projects/Work/hive-2023/.venv/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
survey_raw.columns = ['name', 'lob', 'role', 'location', 'attending', '9 (Q9) ',
       '12 (Q12_1) ', '12 (Q12_2) ', '12 (Q12_3) ', '12 (Q12_4) ',
       '12 (Q12_5) ', '12 (Q12_6) ', '12 (Q12_7) ', '12 (Q12SPECIFIED_6) ',
       'travel_notes', 'doing-talk', 'talk-title', 'talk-description', '18 (Q18) ',
       'lob_rep_interest', '20 (Q20) ', '21 (Q21) ', '23 (Q23) ', 'lob_rep_reason',
       '25 (Q25_1) ', '26 (Q26_1) ', 'topics', 'other', 'Started',
       'Completed', 'Branched Out', 'Over Quota', 'Last Modified']

In [5]:
travel_raw["is_host"] = travel_raw['Location'] == travel_raw['Travel to']
travel = travel_raw[['Location', 'Travel to', 'People', 'Est total travel pp', 'is_host']].dropna().copy()
travel['People'] = travel['People'].astype(int)
travel.columns = ['location', 'destination', 'total_people', 'estimated_travel', 'is_host']
travel['location'] = travel['location'].str.lower()
travel['destination'] = travel['destination'].str.lower()
travel['location'] = travel['location'].str.replace('washington, d.c.', 'washington dc')
travel['destination'] = travel['destination'].str.replace('washington, d.c.', 'washington dc')
travel['destination'] = travel['destination'].str.replace('ã', 'a')
travel['location'] = travel['location'].str.replace('ã', 'a')

In [6]:
survey_raw = survey_raw[survey_raw['location'].str.len() >1]
survey_raw = survey_raw[survey_raw['attending'] == 'Yes']
survey_raw = survey_raw[survey_raw['location'] != 'dfasd']

In [7]:
#clean locations
survey_raw['location'] = survey_raw['location'].str.lower()
survey_raw['location'] = survey_raw['location'].str.replace('chk - ', '')
survey_raw['location'] = survey_raw['location'].str.replace('acc', '')
survey_raw['location'] = survey_raw['location'].str.replace('dal', 'dallas')
survey_raw['location'] = survey_raw['location'].str.replace('guk', 'gurugram')
survey_raw['location'] = survey_raw['location'].str.replace('bos', 'boston')
survey_raw['location'] = survey_raw['location'].str.replace('bostonton', 'boston')
survey_raw['location'] = survey_raw['location'].str.replace('cas', 'casablanca')
survey_raw['location'] = survey_raw['location'].str.replace('casablancaablanca', 'casablanca')
survey_raw['location'] = survey_raw['location'].str.replace(',', '')
survey_raw['location'] = survey_raw['location'].str.replace('-', '')
survey_raw['location'] = survey_raw['location'].str.replace('dallaslaslas', 'dallas')
survey_raw['location'] = survey_raw['location'].str.replace('dallaslas', 'dallas')
survey_raw['location'] = survey_raw['location'].str.replace('á', 'a')
survey_raw['location'] = survey_raw['location'].str.replace('ã', 'a')
survey_raw['location'] = survey_raw['location'].str.replace('ü', 'u')
survey_raw['location'] = survey_raw['location'].str.replace(' india', '')
survey_raw['location'] = survey_raw['location'].str.replace(' south africa', '')
survey_raw['location'] = survey_raw['location'].str.replace(' morocco', '')
survey_raw['location'] = survey_raw['location'].str.replace('gurgaon', 'gurugram')
survey_raw['location'] = survey_raw['location'].str.replace('gurugaon', 'gurugram')
survey_raw['location'] = survey_raw['location'].str.replace('bangalore', 'bengaluru')
survey_raw['location'] = survey_raw['location'].str.replace('  bek', '')
survey_raw['location'] = survey_raw['location'].str.replace('santaigo', 'santiago')
survey_raw['location'] = survey_raw['location'].str.replace(' brazil', '')
survey_raw['location'] = survey_raw['location'].str.replace(' city', '')
survey_raw['location'] = survey_raw['location'].str.replace('sfo', 'san francisco')
survey_raw['location'] = survey_raw['location'].str.replace('was', 'washington dc')
survey_raw['location'] = survey_raw['location'].str.replace('dchington dc', 'dc')
survey_raw['location'] = survey_raw['location'].str.replace('saettle', 'seattle')
survey_raw['location'] = survey_raw['location'].str.replace('colgone', 'cologne')
survey_raw['location'] = survey_raw['location'].str.replace('los angeles', 'temp')
survey_raw['location'] = survey_raw['location'].str.replace('manhattan beach', 'temp')
survey_raw['location'] = survey_raw['location'].str.replace('temp', 'manhattan beach & la')
survey_raw['location'] = survey_raw['location'].str.strip()

In [8]:
unique_locations = survey_raw['location'].unique()
unique_locations.sort()
unique_locations

array(['amsterdam', 'atlanta', 'austin', 'barcelona', 'bek', 'bengaluru',
       'berlin', 'bogota', 'boston', 'brooklyn', 'brussels', 'casablanca',
       'chicago', 'cologne', 'copenhagen', 'dallas', 'denver',
       'dusseldorf', 'frankfurt', 'gurugram', 'helsinki', 'houston',
       'india', 'johannesburg', 'london', 'madrid',
       'manhattan beach & la', 'melbourne', 'miami', 'milan', 'mumbai',
       'munich', 'new delhi', 'new jersey', 'new york', 'nyc', 'oslo',
       'paris', 'perth', 'pittsburgh', 'rio de janeiro', 'san francisco',
       'santiago', 'sao paulo', 'sdasd', 'seattle', 'seoul', 'shanghai',
       'shenzhen', 'silicon valley', 'singapore', 'stockholm', 'sydney',
       'tokyo', 'toronto', 'vienna', 'warsaw', 'washington dc', 'zurich'],
      dtype=object)

# General counts

In [56]:
survey_raw['doing-talk-clean'] =  survey_raw['doing-talk'] ==  'Yes'
survey_raw['doing-talk-clean'] = np.where(survey_raw['doing-talk-clean'] == False, np.nan, survey_raw['doing-talk-clean'])
#survey_raw[['doing-talk-clean', 'doing-talk'] ] 
office_counts = survey_raw.groupby('location').count()[['name', 'doing-talk-clean']]

In [57]:
combined = office_counts.join(travel.set_index('location'), how='left')
combined.columns = ['total_registered', 'total_doing_talk', 'destination', 'total_people', 'estimated_travel', 'is_host']

In [11]:
combined[combined['is_host'] == True].reset_index()

,location,total_registered,total_doing_talk,destination,total_people,estimated_travel,is_host
0,barcelona,7,0,barcelona,27.0,0,True
1,bengaluru,23,2,bengaluru,140.0,0,True
2,berlin,25,7,berlin,51.0,0,True
3,bogota,10,0,bogota,20.0,0,True
4,boston,34,3,boston,97.0,0,True
5,casablanca,13,6,casablanca,43.0,0,True
6,chicago,33,2,chicago,68.0,0,True
7,dallas,4,0,dallas,13.0,0,True
8,frankfurt,5,1,frankfurt,19.0,0,True
9,gurugram,35,5,gurugram,169.0,0,True


In [12]:
combined.to_csv("data/extracted_registrations.csv")

# Travellers list

In [13]:
travel_subset = travel[travel['total_people'] > 1]
merged_with_travel = pd.merge(survey_raw,travel_subset,how='left',left_on='location',right_on='location')
small_office_travellers = merged_with_travel[merged_with_travel['is_host'] == False].copy()

In [15]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

small_office_travellers['estimated_travel_plus_perc'] = small_office_travellers['estimated_travel'] * 1.30
small_office_travellers[['name', 'role', 'lob', 'location', 'destination', 'estimated_travel_plus_perc', 'doing-talk-clean', 'travel_notes', 'other']]

,name,role,lob,location,destination,estimated_travel_plus_perc,doing-talk-clean,travel_notes,other
0,Eivind Strøm,Data Scientist,X AI & Software,oslo,stockholm,364.0,NaN,NaN,.
1,Valentina Jarovaja,Data & Analytics Lead,X Delivery,dusseldorf,frankfurt,546.0,NaN,NaN,NaN
3,Marion Nöske,Senior Director Data & Analytics - X Delivery,X Delivery,dusseldorf,frankfurt,546.0,NaN,NaN,thank you for organizing!
5,Michael Logen,Data & Analytics Lead,X Delivery,dusseldorf,frankfurt,546.0,NaN,NaN,-
19,Martin Schleicher,Data & Analytics Lead – X Delivery,X Delivery,dusseldorf,frankfurt,546.0,NaN,NaN,Don't make optional questions mandatory.
24,Otto Saikkonen,Senior Data Scientist,X AI & Software,helsinki,stockholm,403.0,NaN,NaN,No
30,Shon Little,Software Engineering Senior Manager - X Delivery,X Delivery,manhattan beach & la,manhattan beach,0.0,NaN,NaN,No
31,Dante Cicco,Senior Engineer,X AI & Software,pittsburgh,washington dc,911.3,NaN,No,Nope!
35,Jo Saakvitne,Lead Data Scientist,X AI & Software,oslo,stockholm,364.0,NaN,No,No
39,Petter Moen,Senior Data Scientist,X AI & Software,oslo,stockholm,364.0,NaN,NaN,NaN


# Host office representation

In [16]:
lob_counts = merged_with_travel.groupby(['destination', 'lob']).count()['name'].reset_index()
lob_counts_pivoted = pd.pivot(lob_counts, columns=['lob'], index=['destination'])['name'][['X AI & Software', 'X Delivery', 'X Ventures']]

In [17]:
lob_counts_pivoted

lob,X AI & Software,X Delivery,X Ventures
destination,,,
barcelona,3.0,4.0,NaN
bengaluru,3.0,22.0,NaN
berlin,11.0,2.0,20.0
bogota,1.0,9.0,NaN
boston,17.0,17.0,NaN
casablanca,1.0,11.0,1.0
chicago,10.0,22.0,1.0
dallas,14.0,1.0,1.0
frankfurt,2.0,11.0,NaN


In [18]:
lob_rep_plan = pd.read_excel('lob_rep_plan.xlsx') 
lob_rep_plan['Missing LOB'] = lob_rep_plan['Missing LOB'].str.split(',')
lob_rep_plan = lob_rep_plan.explode(['Missing LOB'])

# LOB reps

In [19]:
lob_reps = survey_raw[(~survey_raw['lob_rep_interest'].isna()) & (survey_raw['lob_rep_interest'] != 'None of the above')]


In [20]:
lob_reps.to_csv("data/extracted_lob_reps.csv")

### AI & software

In [21]:
lob_counts_pivoted[lob_counts_pivoted['X AI & Software'].isna()]

lob,X AI & Software,X Delivery,X Ventures
destination,,,


In [22]:
lob_reps[lob_reps['lob'] == 'X AI & Software'].dropna(subset=['lob_rep_reason'])

,name,lob,role,location,attending,9 (Q9),12 (Q12_1),12 (Q12_2),12 (Q12_3),12 (Q12_4),12 (Q12_5),12 (Q12_6),12 (Q12_7),12 (Q12SPECIFIED_6),travel_notes,doing-talk,talk-title,talk-description,18 (Q18),lob_rep_interest,20 (Q20),21 (Q21),23 (Q23),lob_rep_reason,25 (Q25_1),26 (Q26_1),topics,other,Started,Completed,Branched Out,Over Quota,Last Modified,doing-talk-clean
Record ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
638295193090591825,Guillem Borrell Nogueras,X AI & Software,Lead AI Engineer,madrid,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,Yes,All you need is Duckdb,Duckdb is a relatively new high-performance qu...,45 minutes,I am interested in being LOB representative in...,NaN,No,NaN,I've been teaching technical stuff to DS and S...,NaN,NaN,Anything that is interesting and not about GenAI,"Please, don't make it only about GenAI",2023-09-05 14:01:49,2023-09-05 14:13:08,NaN,NaN,2023-09-05 14:13:08,1.0
638298051723847524,Archit Matta,X AI & Software,Data Scientist,new york,Yes,No,-,-,-,-,-,"Others ( please specify, including allergies)",-,No Beef,NaN,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,Yes,NaN,I'm super excited and have tried to contribute...,NaN,NaN,I'd be interested in knowing about more networ...,I would also be super interested in helping pl...,2023-09-08 21:26:12,2023-09-08 21:34:28,NaN,NaN,2023-09-08 21:34:28,NaN
638301162078151552,Abby Gong,X AI & Software,Senior Data Scientist,boston,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,Yes,MedTech supply chain AI capabilities,The application of supply chain AI capabilitie...,15 minutes,I am interested in being LOB representative in...,NaN,No,NaN,"I’m from AI&Software, but I have done multiple...",NaN,NaN,"Future one BCGx offering, collaborative ways o...",would like to hear the envision from org team ...,2023-09-12 11:50:07,2023-09-12 12:01:12,NaN,NaN,2023-09-12 19:37:38,1.0
638301202864066433,Kamal Darchini,X AI & Software,Lead data scientist,toronto,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,No,NaN,I have been in this LOB for 3+ years and have ...,NaN,NaN,Hands on GenAI,Thanks for organizing!,2023-09-12 12:58:06,2023-09-12 13:11:28,NaN,NaN,2023-09-12 13:11:28,NaN
638301255487576553,Saurabh Mahajan,X AI & Software,AI Engineer,denver,Yes,Yes,-,Vegetarian,-,-,-,-,-,NaN,NaN,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,No,NaN,Denver office has a very small presence of gam...,NaN,NaN,-,Can you send a broad categories and informatio...,2023-09-12 14:25:48,2023-09-12 14:40:44,NaN,NaN,2023-09-12 14:40:44,NaN
638312276035642564,Olga Kotova,X AI & Software,Data Scientist,london,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,Yes,Transforming Drug Discovery workflow using AI/...,Global Biopharma company was looking to accele...,30 minutes,I am interested in being LOB representative in...,NaN,No,NaN,I have strong public speaking skills emerging ...,NaN,NaN,Generative algorithms and language models,-,2023-09-25 08:33:23,2023-09-25 08:45:57,NaN,NaN,2023-09-25 08:45:57,1.0
638315193987323543,Kaoru Schwarzenegger,X AI & Software,Data Scientist,tokyo,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,No,NaN,Since I have a wide field of interest apart fr...,NaN,NaN,Different perspectives from other LOBs and ide...,Will likely not be able to block the whole day...,2023-09-28 17:36:38,2023-09-28 17:43:25,NaN,NaN,2023-09-28 17:43:25,NaN
638315202876045630,Adam Benali,X AI & Software,AI Engineer,stockholm,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,No,NaN,"We pretty much only have one LOB in Stockholm,...",NaN,NaN,-,-,2023-09-28 17:51:27,2023-09-28 17:57:33,NaN,NaN,2023-09-28 17:57:33,NaN
638315430336436294,Dave Rattan,X AI & Software,AI Engineer,sydney,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,Yes,How to mo

### X Delivery

In [23]:
lob_counts_pivoted[lob_counts_pivoted['X Delivery'].isna()]

lob,X AI & Software,X Delivery,X Ventures
destination,,,
milan,15.0,NaN,NaN
paris,33.0,NaN,NaN
santiago,11.0,NaN,NaN
singapore,14.0,NaN,8.0
sydney,27.0,NaN,3.0
tokyo,7.0,NaN,3.0
washington dc,9.0,NaN,NaN
zurich,7.0,NaN,NaN


In [52]:
len(lob_counts_pivoted[lob_counts_pivoted['X Delivery'].isna()].index)

8

In [24]:
lob_reps[lob_reps['lob'] == 'X Delivery'].dropna(subset=['lob_rep_reason'])

,name,lob,role,location,attending,9 (Q9),12 (Q12_1),12 (Q12_2),12 (Q12_3),12 (Q12_4),12 (Q12_5),12 (Q12_6),12 (Q12_7),12 (Q12SPECIFIED_6),travel_notes,doing-talk,talk-title,talk-description,18 (Q18),lob_rep_interest,20 (Q20),21 (Q21),23 (Q23),lob_rep_reason,25 (Q25_1),26 (Q26_1),topics,other,Started,Completed,Branched Out,Over Quota,Last Modified,doing-talk-clean
Record ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
638298890718873054,Amine Benaddi,X Delivery,Engineering Lead,casablanca,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,Yes,Leveraging typescript for micro service,"Trpc, turbo, nodejs",30 minutes,I am interested in being LOB representative in...,NaN,No,NaN,Interesting topic for next projects,NaN,NaN,Architectures leveraged in other cases,"Comparing Astro, Nextjs, fresh, blitzjs, etc",2023-09-09 20:44:31,2023-09-09 20:49:39,NaN,NaN,2023-09-09 20:49:39,1.0
638307984685797820,Panagiota Papakosta,X Delivery,Senior Analyst,munich,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,Yes,GeoAnalytics to accelerate Climate & Sustainab...,what is understood by the science of where and...,15 minutes,I am interested in being LOB representative in...,NaN,Yes,NaN,happy to represent LOB in the Munich office,NaN,NaN,other C&S applications,thanks for orga,2023-09-20 09:21:08,2023-09-20 09:27:17,NaN,NaN,2023-09-20 09:27:17,1.0
638309142512518582,Purnima Aggarwal,X Delivery,Lead data analyst,new delhi,Yes,Yes,Vegan,Vegetarian,-,-,-,"Others ( please specify, including allergies)",-,No onion and no garlic,No,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,Yes,NaN,Not sure,NaN,NaN,Outlook,No,2023-09-21 17:30:51,2023-09-21 17:33:52,NaN,NaN,2023-09-21 17:34:01,NaN
638309193066578398,Amar Patel,X Delivery,Product Management Director,austin,Yes,Yes,-,-,-,-,-,-,No dietary restrictions,NaN,No,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,Yes,NaN,I interface across all the areas within the X ...,NaN,NaN,GenAI use cases that we have developed and dep...,no,2023-09-21 18:55:06,2023-09-21 19:00:59,NaN,NaN,2023-09-21 19:00:59,NaN
638309418719094351,Jason He,X Delivery,Senior Delivery Analyst,shenzhen,Yes,Yes,-,-,-,-,-,-,No dietary restrictions,NaN,No,Yes,"OrgBuilder, data-driven organizational redesig...",Showcase and demo on OrgBuilder tool,30 minutes,I am interested in being LOB representative in...,NaN,No,NaN,There are only 2 X Delivery people in Greater ...,NaN,NaN,Sharing (case / tool) from Venture team,No,2023-09-22 01:11:11,2023-09-22 01:20:59,NaN,NaN,2023-09-22 01:20:59,1.0
638312379032342134,Oliver Hof,X Delivery,Technical Delivery Sr. Manager,warsaw,Yes,Yes,-,-,-,-,-,-,-,NaN,No,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,Yes,NaN,Almost 8 years in BCG; have been in the same j...,NaN,NaN,How teams are using AI tools internally. Any r...,"I am not quite sure how the LOB Rep will work,...",2023-09-25 11:25:03,2023-09-25 11:30:19,NaN,NaN,2023-09-25 11:30:19,NaN
638314001137590077,mohd sharique ansari,X Delivery,engineering lead,gurugram,Yes,No,Vegan,Vegetarian,-,-,-,-,-,NaN,NaN,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,Yes,NaN,I am part of the most revenue generating produ...,NaN,NaN,"Software Security, Infra modernization techniques",no,2023-09-27 08:28:33,2023-09-27 08:37:48,NaN,NaN,2023-09-27 08:37:48,NaN
638314079246651573,Madhur Mukhi,X Delivery,Engineering Lead,gurugram,Yes,No,-,Vegetarian,-,-,-,-,-,NaN,NaN,Yes,How we brought a legacy product in a Multi ten...,Workspace Manager is a utility to host and man...,15 minutes,I am interested in being LOB representative in...,NaN,Yes,NaN,Yes,NaN,NaN,"Agile , Software development",NaN,2023-09-27 10:38:44,2023-09-27 10:52:54,NaN,NaN,2023-09-27 10:52:54,1.0
638315013930037917,Luísa Lichfett,X Delivery,Software Engineer,sao paulo,Yes,Yes,-,-,-,-,-,-,No dietary restrictions,NaN,Not that I know of,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,No,NaN,I believe that a LOB should have some pa

# Options for LOB Rep X.Delivery

- Amine Benaddi -> Paris or Zurich
- Panagiota Papakosta -> Paris or Zurich or Milan
- Oliver Hof -> Zurich
- David Hoyos -> Santiago
- Deepak Trehan -> Washington DC
- Jason He -> Tokyo
- Sukrit Kurle -> Singapore

In [48]:
travel_cmn_cdg = 330
travel_cmn_zrh = 600 #takes too long vs cdg


travel_muc_cdg = 260
travel_muc_zrh = 450
travel_muc_mxp = 400

travel_waw_zrh = 250

hotel_mxp = 200
hotel_zrh = 200
hotel_cdg = 300


travel_bog_scl = 11000 #7hr flight, lol
hotel_scl = 400 #2 nights required

travel_bos_iad = 200
hotel_iad = 400

travel_szx_tok = 2100
hotel_tok = 300

travel_blr_sin = 2100 #5hr flight
hotel_sin = 600 #2 nights required

total_cmn_cdg = travel_cmn_cdg + hotel_cdg
total_cmn_zrh = travel_cmn_zrh + hotel_zrh

total_muc_cdg = travel_muc_cdg + hotel_cdg
total_muc_zrh = travel_muc_zrh + hotel_zrh
total_muc_mxp = travel_muc_mxp + hotel_mxp

total_scl = travel_bog_scl + hotel_scl
total_iad = travel_bos_iad + hotel_iad
total_tok = travel_szx_tok + hotel_tok
total_sin = travel_blr_sin + hotel_sin
total_waw_zrh = travel_waw_zrh + hotel_zrh


travel_lob_rep_delivery = lob_reps[lob_reps['lob'] == 'X Delivery'].dropna(subset=['lob_rep_reason'])
travel_lob_rep_delivery = travel_lob_rep_delivery.loc[travel_lob_rep_delivery['name'].isin(['Amine Benaddi', 'Panagiota Papakosta', 'Oliver Hof', 'David Hoyos', 'Deepak Trehan', 'Jason He', 'Sukrit Kurle'])]
travel_lob_rep_delivery = travel_lob_rep_delivery.drop_duplicates(subset='name')

travel_lob_rep_delivery.insert(4, 'destination', ['paris', 'milan', 'tokyo', 'zurich', 'singapore', 'washington', 'santiago'], True)
travel_lob_rep_delivery.insert(5, 'estimated_travel_plus_perc', [total_cmn_cdg, total_muc_mxp, total_tok, total_waw_zrh, total_sin, total_iad, total_scl], True)


travel_lob_rep_delivery[['name', 'lob', 'location', 'destination', 'estimated_travel_plus_perc', 'doing-talk-clean']]



,name,lob,location,destination,estimated_travel_plus_perc,doing-talk-clean
Record ID,,,,,,
638298890718873054,Amine Benaddi,X Delivery,casablanca,paris,630,1.0
638307984685797820,Panagiota Papakosta,X Delivery,munich,milan,600,1.0
638309418719094351,Jason He,X Delivery,shenzhen,tokyo,2400,1.0
638312379032342134,Oliver Hof,X Delivery,warsaw,zurich,450,NaN
638315943002284580,Sukrit Kurle,X Delivery,bengaluru,singapore,2700,NaN
638315943121595054,Deepak Trehan,X Delivery,boston,washington,600,1.0
638316005914858606,David Hoyos,X Delivery,bogota,santiago,11400,NaN


# Ventures

In [53]:
lob_counts_pivoted[lob_counts_pivoted['X Ventures'].isna()]

lob,X AI & Software,X Delivery,X Ventures
destination,,,
barcelona,3.0,4.0,NaN
bengaluru,3.0,22.0,NaN
bogota,1.0,9.0,NaN
boston,17.0,17.0,NaN
frankfurt,2.0,11.0,NaN
gurugram,2.0,37.0,NaN
madrid,9.0,14.0,NaN
milan,15.0,NaN,NaN
munich,7.0,4.0,NaN


In [54]:
len(lob_counts_pivoted[lob_counts_pivoted['X Ventures'].isna()].index)

21

In [26]:
lob_reps[lob_reps['lob'] == 'X Ventures'].dropna(subset=['lob_rep_reason'])

,name,lob,role,location,attending,9 (Q9),12 (Q12_1),12 (Q12_2),12 (Q12_3),12 (Q12_4),12 (Q12_5),12 (Q12_6),12 (Q12_7),12 (Q12SPECIFIED_6),travel_notes,doing-talk,talk-title,talk-description,18 (Q18),lob_rep_interest,20 (Q20),21 (Q21),23 (Q23),lob_rep_reason,25 (Q25_1),26 (Q26_1),topics,other,Started,Completed,Branched Out,Over Quota,Last Modified,doing-talk-clean
Record ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
638301182426033496,Robin Weston,X Ventures,Principal,singapore,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,Yes,Building Gen AI PRODUCTS,Who decided that Gen AI doesn't require and de...,15 minutes,I am interested in being LOB representative in...,NaN,No,NaN,I am going to be in London already visiting fa...,NaN,NaN,Would love to hear about some ventures - I hav...,NaN,2023-09-12 12:24:02,2023-09-12 12:28:14,NaN,NaN,2023-09-12 12:28:14,1.0
638315196919892165,Sebastian Geidies,X Ventures,"Partner & Vice President, Engineering",berlin,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,No,NaN,Long experience with X.Ventures,NaN,NaN,Distributed Computing,NaN,2023-09-28 17:41:31,2023-09-28 17:43:46,NaN,NaN,2023-09-28 17:43:46,NaN
638315201269934155,Astera Schneeweisz,X Ventures,VP of Engineering,berlin,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,No,NaN,"Because i often work in the role of a GM, and ...",NaN,NaN,"Diversity, cultural topics, eng starter projects",not at this time :),2023-09-28 17:48:46,2023-09-28 17:56:42,NaN,NaN,2023-09-28 17:56:42,NaN
638315258771559218,Constantine Karlis,X Ventures,Lead Engineer,berlin,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,Yes,"Privacy, Enterprise and DLT",How we found the right balance of openness and...,30 minutes,I am interested in being LOB representative in...,NaN,No,NaN,I've been at DV/X Ventures for a very long tim...,NaN,NaN,"In general: cats, noodles, synthesisers. Speci...",For the talk I would like to collaborate with ...,2023-09-28 19:24:37,2023-09-28 19:44:53,NaN,NaN,2023-09-28 19:44:53,1.0


# Options for LOB Rep X.Ventures

- Robin Weston -> Shanghai
- Sebastian Geidies, Astera, Constantine -> Munich, Frankfurt, Milan, Zurich or Stockholm

In [55]:
travel_sin_pvg = 1000 #estimate: can't get through travel portal

hotel_pvg = 260

travel_ber_muc_flight = 330
travel_ber_muc_train = 450 #more like 200 on DB directly

travel_ber_fra_flight = 260 
travel_ber_fra_train = 590 #more like 200 on DB directly

travel_ber_mxp = 300
travel_ber_zrh = 420
travel_ber_arn = 200

hotel_muc = 166
hotel_fra = 130
hotel_mxp = 200
hotel_zrh = 200
hotel_arn = 150

total_pvg = travel_sin_pvg + hotel_pvg
total_muc = travel_ber_muc_flight + hotel_muc
total_fra = travel_ber_fra_flight + hotel_fra
total_mxp = travel_ber_mxp + hotel_mxp
total_zrh = travel_ber_zrh + hotel_zrh
total_arn = travel_ber_arn + hotel_arn

travel_lob_rep_ventures = lob_reps[lob_reps['lob'] == 'X Ventures'].dropna(subset=['lob_rep_reason'])


travel_lob_rep_ventures.insert(4, 'destination', ['shanghai', 'munich', 'frankfurt', 'stockholm'], True)
travel_lob_rep_ventures.insert(5, 'estimated_travel_plus_perc', [total_pvg, total_muc, total_fra, total_arn], True)


travel_lob_rep_ventures[['name', 'lob', 'location', 'destination', 'estimated_travel_plus_perc', 'doing-talk-clean']]



,name,lob,location,destination,estimated_travel_plus_perc,doing-talk-clean
Record ID,,,,,,
638301182426033496,Robin Weston,X Ventures,singapore,shanghai,1260,1.0
638315196919892165,Sebastian Geidies,X Ventures,berlin,munich,496,NaN
638315201269934155,Astera Schneeweisz,X Ventures,berlin,frankfurt,390,NaN
638315258771559218,Constantine Karlis,X Ventures,berlin,stockholm,350,1.0


In [64]:
talks = survey_raw[['name', 'location', 'talk-title', 'talk-description', 'doing-talk-clean', 'doing-talk', '18 (Q18) ']]
talks = talks[talks['doing-talk-clean'] == 1]

# Define a dictionary to map cities to regions
city_to_region = {
    'amsterdam': 'EMESA',
    'atlanta': 'NAMR',
    'austin': 'NAMR',
    'barcelona': 'EMESA',
    'bek': 'EMESA',
    'bengaluru': 'APAC',
    'berlin': 'EMESA',
    'bogota': 'NAMR',
    'boston': 'NAMR',
    'brooklyn': 'NAMR',
    'brussels': 'EMESA',
    'casablanca': 'EMESA',
    'chicago': 'NAMR',
    'cologne': 'EMESA',
    'copenhagen': 'EMESA',
    'dallas': 'NAMR',
    'denver': 'NAMR',
    'dusseldorf': 'EMESA',
    'frankfurt': 'EMESA',
    'gurugram': 'APAC',
    'helsinki': 'EMESA',
    'houston': 'NAMR',
    'india': 'APAC',
    'johannesburg': 'EMESA',
    'london': 'EMESA',
    'madrid': 'EMESA',
    'manhattan beach & la': 'NAMR',
    'melbourne': 'APAC',
    'miami': 'NAMR',
    'milan': 'EMESA',
    'mumbai': 'APAC',
    'munich': 'EMESA',
    'new delhi': 'APAC',
    'new jersey': 'NAMR',
    'new york': 'NAMR',
    'nyc': 'NAMR',
    'oslo': 'EMESA',
    'paris': 'EMESA',
    'perth': 'APAC',
    'pittsburgh': 'NAMR',
    'rio de janeiro': 'NAMR',
    'san francisco': 'NAMR',
    'santiago': 'NAMR',
    'sao paulo': 'NAMR',
    'seattle': 'NAMR',
    'seoul': 'APAC',
    'shanghai': 'APAC',
    'shenzhen': 'APAC',
    'silicon valley': 'NAMR',
    'singapore': 'APAC',
    'stockholm': 'EMESA',
    'sydney': 'APAC',
    'tokyo': 'APAC',
    'toronto': 'NAMR',
    'vienna': 'EMESA',
    'warsaw': 'EMESA',
    'washington dc': 'NAMR',
    'zurich': 'EMESA'
}

talks['region'] = talks['location'].map(city_to_region)

talks.sort_values(by=['region'])



,name,location,talk-title,talk-description,doing-talk-clean,doing-talk,18 (Q18),region
Record ID,,,,,,,,
638314079246651573,Madhur Mukhi,gurugram,How we brought a legacy product in a Multi ten...,Workspace Manager is a utility to host and man...,1.0,Yes,15 minutes,APAC
638309418719094351,Jason He,shenzhen,"OrgBuilder, data-driven organizational redesig...",Showcase and demo on OrgBuilder tool,1.0,Yes,30 minutes,APAC
638303525517533262,Jurgen Hanekom,perth,Modern Local Data Science Tech Stack,Combination of third party tools to solve a lo...,1.0,Yes,30 minutes,APAC
638303510249701567,Michael Chu,sydney,NaN,NaN,1.0,Yes,NaN,APAC
638303491659255801,Adam Whybrew,sydney,ShopGTT our patent-pending transformer-based s...,We've applied for a patent on using Trabsforme...,1.0,Yes,15 minutes,APAC
638309631694877374,Sammy Garland,singapore,NaN,NaN,1.0,Yes,NaN,APAC
638302636540372376,Gaurav Gaurav,gurugram,Navigating the Future: Blockchain and Hyperled...,"Discover how Hyperledger Fabric, an open-sourc...",1.0,Yes,15 minutes,APAC
638311053147961162,Aparna Kapoor,singapore,NaN,NaN,1.0,Yes,NaN,APAC
638302198936398511,Gaurav Gaurav,gurugram,NaN,NaN,1.0,Yes,NaN,APAC
